In [14]:
import fastparquet
import pyarrow
import pandas as pd
import numpy as np

In [16]:
df = pd.read_parquet("yellow_trip_data.parquet", engine = "pyarrow")

ArrowInvalid: Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
# Important Feature's Imformation

# Target: Duration
# Model: Regression Model
# Independent features: trip_distance, total_amount, PULocationID, DOLocationID, pickup_time -> (which have to converted into time_sin and time_cos)  

In [ ]:
df["duration"] = df["lpep_dropoff_datetime"] - df["lpep_pickup_datetime"]
df["duration"] = df["duration"].astype("str")
df.loc[0, "duration"]

In [ ]:
def getMins(x):
    h, m, s = list(map(int, x.split()[2].split(":")))
    return h*60 + m + s // 60

In [ ]:
df["duration"] = df["duration"].apply(lambda x: getMins(x))

In [ ]:
df["hour"] = df["lpep_pickup_datetime"].dt.hour
df["minute"] = df["lpep_pickup_datetime"].dt.minute
df["time_numeric"] = df["hour"] + df["minute"] / 60

In [ ]:
df["time_sin"] = np.sin(2 * np.pi * df["time_numeric"] / 24)
df["time_cos"] = np.cos(2 * np.pi * df["time_numeric"] / 24)

In [ ]:
nominal_cols = ["PULocationID", "DOLocationID"]
numerical_cols = ["time_sin", "time_cos", "trip_distance", "total_amount"]

df = df[numerical_cols + nominal_cols + ["duration"]]

In [ ]:
df.head()

In [ ]:
X = df.drop("duration", axis = 1)
y = df["duration"]

In [ ]:
# models -> Xgboost, random_forest, linear_regression, knn

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error

## LinearRegression Model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
preprocess = ColumnTransformer([
    ("cat", OneHotEncoder(sparse_output = True, handle_unknown = "ignore"), nominal_cols),
    ("num", StandardScaler(), numerical_cols)
])

In [ ]:
LR_model = Pipeline([
    ("preprocess", preprocess),
    ("model", LinearRegression())
])

In [ ]:
LR_model.fit(X_train, y_train)

In [ ]:
print("Training Score:", LR_model.score(X_train, y_train))
print("Testing Score:", LR_model.score(X_test, y_test))